In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
#tf.contrib.rnn.DropoutWrapper -> tf.nn.rnn_cell.DropoutWrapper
#tf.contrib.layers.xavier_initializer -> tf.keras.initializers.glorot_uniform
#tf.contrib.rnn.MultiRNNCell -> tf.nn.rnn_cell.MultiRNNCell

In [2]:
#if __name__ == '__main__':
#    tf.logging.set_verbosity(tf.logging.INFO)
#    tf.app.run(main)

# config.py

In [3]:
#*- coding: utf-8 -*-
#import tensorflow as tf

tf.app.flags.DEFINE_string('f', '', 'kernel') # 주피터에서 커널에 전달하기 위한 프레그 방법
tf.app.flags.DEFINE_integer('batch_size', 64, 'batch size') # 배치 크기
tf.app.flags.DEFINE_integer('train_steps', 20000, 'train steps') # 학습 에포크
tf.app.flags.DEFINE_float('dropout_width', 0.8, 'dropout width') # 드롭아웃 크기
tf.app.flags.DEFINE_integer('layer_size', 1, 'layer size') # 멀티 레이어 크기 (multi rnn)
tf.app.flags.DEFINE_integer('hidden_size', 128, 'weights size') # 가중치 크기
tf.app.flags.DEFINE_float('learning_rate', 1e-3, 'learning rate') # 학습률
tf.app.flags.DEFINE_float('teacher_forcing_rate', 0.7, 'teacher forcing rate') # 학습시 디코더 인풋 정답 지원율
tf.app.flags.DEFINE_string('data_path', '../data_in/ChatBotData.csv', 'data path') #  데이터 위치
tf.app.flags.DEFINE_string('vocabulary_path', './data_out/vocabularyData.voc', 'vocabulary path') # 사전 위치
tf.app.flags.DEFINE_string('check_point_path', './data_out/check_point', 'check point path') # 체크 포인트 위치
tf.app.flags.DEFINE_string('save_model_path', './data_out/model', 'save model') # 모델 저장 경로
tf.app.flags.DEFINE_integer('shuffle_seek', 1000, 'shuffle random seek') # 셔플 시드값
tf.app.flags.DEFINE_integer('max_sequence_length', 25, 'max sequence length') # 시퀀스 길이
tf.app.flags.DEFINE_integer('embedding_size', 128, 'embedding size') # 임베딩 크기
tf.app.flags.DEFINE_boolean('embedding', True, 'Use Embedding flag') # 임베딩 유무 설정
tf.app.flags.DEFINE_boolean('multilayer', True, 'Use Multi RNN Cell') # 멀티 RNN 유무
tf.app.flags.DEFINE_boolean('attention', True, 'Use Attention') #  어텐션 사용 유무
tf.app.flags.DEFINE_boolean('teacher_forcing', True, 'Use Teacher Forcing') # 학습시 디코더 인풋 정답 지원 유무
tf.app.flags.DEFINE_boolean('tokenize_as_morph', False, 'set morph tokenize') # 형태소에 따른 토크나이징 사용 유무
tf.app.flags.DEFINE_boolean('serving', False, 'Use Serving') #  서빙 기능 지원 여부
tf.app.flags.DEFINE_boolean('loss_mask', False, 'Use loss mask') # PAD에 대한 마스크를 통한 loss를 제한

# Define FLAGS
DEFINES = tf.app.flags.FLAGS

# data.py

In [4]:
# -*- coding: utf-8 -*-
from konlpy.tag import Okt
import pandas as pd
#import tensorflow as tf
import enum
import os
import re
from sklearn.model_selection import train_test_split
#import numpy as np
#from configs import DEFINES

from tqdm import tqdm

In [5]:
PAD_MASK = 0
NON_PAD_MASK = 1

FILTERS = "([~.,!?\"':;)(])"
PAD = "<PADDING>"
STD = "<START>"
END = "<END>"
UNK = "<UNKNOWN>"

PAD_INDEX = 0
STD_INDEX = 1
END_INDEX = 2
UNK_INDEX = 3

MARKER = [PAD, STD, END, UNK]
CHANGE_FILTER = re.compile(FILTERS)

In [6]:
def load_data():
    # 판다스를 통해서 데이터를 불러온다.
    data_df = pd.read_csv(DEFINES.data_path, header=0)
    # 질문과 답변 열을 가져와 question과 answer에 넣는다.
    question, answer = list(data_df['Q']), list(data_df['A'])
    # skleran에서 지원하는 함수를 통해서 학습 셋과 
    # 테스트 셋을 나눈다.
    train_input, eval_input, train_label, eval_label = train_test_split(question, answer, test_size=0.33, random_state=42)
    # 그 값을 리턴한다.
    return train_input, train_label, eval_input, eval_label


In [7]:
def prepro_like_morphlized(data):
    # 형태소 분석 모듈 객체를
    # 생성합니다.

    morph_analyzer = Okt()
    # 형태소 토크나이즈 결과 문장을 받을
    #  리스트를 생성합니다.
    result_data = list()
    # 데이터에 있는 매 문장에 대해 토크나이즈를
    # 할 수 있도록 반복문을 선언합니다.
    for seq in tqdm(data):
        # Twitter.morphs 함수를 통해 토크나이즈 된
        # 리스트 객체를 받고 다시 공백문자를 기준으로
        # 하여 문자열로 재구성 해줍니다.
        morphlized_seq = " ".join(morph_analyzer.morphs(seq.replace(' ', '')))
        result_data.append(morphlized_seq)

    return result_data

In [8]:
# 인덱스화 할 value와 키가 워드이고 
# 값이 인덱스인 딕셔너리를 받는다.
def enc_processing(value, dictionary):
    # 인덱스 값들을 가지고 있는 
    # 배열이다.(누적된다.)
    sequences_input_index = []
    # 하나의 인코딩 되는 문장의 
    # 길이를 가지고 있다.(누적된다.)
    sequences_length = []
    # 형태소 토크나이징 사용 유무
    if DEFINES.tokenize_as_morph:
        value = prepro_like_morphlized(value)

    # 한줄씩 불어온다.
    for sequence in value:
        # FILTERS = "([~.,!?\"':;)(])"
        # 정규화를 사용하여 필터에 들어 있는 
        # 값들을 "" 으로 치환 한다.
        sequence = re.sub(CHANGE_FILTER, "", sequence)
        # 하나의 문장을 인코딩 할때 
        # 가지고 있기 위한 배열이다.
        sequence_index = []
        # 문장을 스페이스 단위로 
        # 자르고 있다.
        for word in sequence.split():
            # 잘려진 단어들이 딕셔너리에 존재 하는지 보고 
            # 그 값을 가져와 sequence_index에 추가한다.
            if dictionary.get(word) is not None:
                sequence_index.extend([dictionary[word]])
            # 잘려진 단어가 딕셔너리에 존재 하지 않는 
            # 경우 이므로 UNK(2)를 넣어 준다.
            else:
                sequence_index.extend([dictionary[UNK]])
        # 문장 제한 길이보다 길어질 경우 뒤에 토큰을 자르고 있다.
        if len(sequence_index) > DEFINES.max_sequence_length:
            sequence_index = sequence_index[:DEFINES.max_sequence_length]
        # 하나의 문장에 길이를 넣어주고 있다.
        sequences_length.append(len(sequence_index))
        # max_sequence_length보다 문장 길이가 
        # 작다면 빈 부분에 PAD(0)를 넣어준다.
        sequence_index += (DEFINES.max_sequence_length - len(sequence_index)) * [dictionary[PAD]]
        # 뒤로 넣어 준다. 
        sequence_index.reverse()
        # 인덱스화 되어 있는 값을 
        # sequences_input_index에 넣어 준다.
        sequences_input_index.append(sequence_index)
    # 인덱스화된 일반 배열을 넘파이 배열로 변경한다. 
    # 이유는 텐서플로우 dataset에 넣어 주기 위한 
    # 사전 작업이다.
    # 넘파이 배열에 인덱스화된 배열과 
    # 그 길이를 넘겨준다.  
    return np.asarray(sequences_input_index), sequences_length


In [9]:
# 인덱스화 할 value와 키가 워드 이고
# 값이 인덱스인 딕셔너리를 받는다.
def dec_target_processing(value, dictionary):
    # 인덱스 값들을 가지고 있는 
    # 배열이다.(누적된다)
    sequences_target_index = []
    sequences_length = []
    # 형태소 토크나이징 사용 유무
    if DEFINES.tokenize_as_morph:
        value = prepro_like_morphlized(value)
    # 한줄씩 불어온다.
    for sequence in value:
        # FILTERS = "([~.,!?\"':;)(])"
        # 정규화를 사용하여 필터에 들어 있는 
        # 값들을 "" 으로 치환 한다.
        sequence = re.sub(CHANGE_FILTER, "", sequence)
        # 문장에서 스페이스 단위별로 단어를 가져와서 
        # 딕셔너리의 값인 인덱스를 넣어 준다.
        # 디코딩 출력의 마지막에 END를 넣어 준다.
        sequence_index = [dictionary[word] for word in sequence.split()]
        # 문장 제한 길이보다 길어질 경우 뒤에 토큰을 자르고 있다.
        # 그리고 END 토큰을 넣어 준다
        if len(sequence_index) >= DEFINES.max_sequence_length:
            sequence_index = sequence_index[:DEFINES.max_sequence_length-1] + [dictionary[END]]
        else:
            sequence_index += [dictionary[END]]
        
        # 학습시 PAD 마스크를 위한 벡터를 구성한다.                   
        sequences_length.append([PAD_MASK if num > len(sequence_index) else NON_PAD_MASK for num in range (DEFINES.max_sequence_length)])
        # max_sequence_length보다 문장 길이가 
        # 작다면 빈 부분에 PAD(0)를 넣어준다.
        sequence_index += (DEFINES.max_sequence_length - len(sequence_index)) * [dictionary[PAD]]
        # 인덱스화 되어 있는 값을 
        # sequences_target_index에 넣어 준다.
        sequences_target_index.append(sequence_index)
    # 인덱스화된 일반 배열을 넘파이 배열로 변경한다. 
    # 이유는 텐서플로우 dataset에 넣어 주기 위한 사전 작업이다.
    # 넘파이 배열에 인덱스화된 배열과 그 길이를 넘겨준다.
    return np.asarray(sequences_target_index), np.asarray(sequences_length)


In [10]:
# 인덱스를 스트링으로 변경하는 함수이다.
# 바꾸고자 하는 인덱스 value와 인덱스를 
# 키로 가지고 있고 값으로 단어를 가지고 있는 
# 딕셔너리를 받는다.
def pred2string(value, dictionary):
    # 텍스트 문장을 보관할 배열을 선언한다.
    sentence_string = []
    # 인덱스 배열 하나를 꺼내서 v에 넘겨준다.
    if DEFINES.serving == True:
        for v in value['output']: 
            sentence_string = [dictionary[index] for index in v]
    else:
        for v in value:
            # 딕셔너리에 있는 단어로 변경해서 배열에 담는다.
            sentence_string = [dictionary[index] for index in v['indexs']]
    
    print(sentence_string)
    answer = ""
    # 패딩값도 담겨 있으므로 패딩은 모두 스페이스 처리 한다.
    for word in sentence_string:
        if word not in PAD and word not in END:
            answer += word
            answer += " "
    # 결과를 출력한다.
    print(answer)
    return answer

In [11]:
def rearrange(input, target):
    features = {"input": input}
    return features, target


In [12]:
def train_rearrange(input, length, target):
    features = {"input": input, "length": length}
    return features, target

In [13]:
# 학습에 들어가 배치 데이터를 만드는 함수이다.
def train_input_fn(train_input_enc, train_target_dec_length, train_target_dec, batch_size):
    # Dataset을 생성하는 부분으로써 from_tensor_slices부분은 
    # 각각 한 문장으로 자른다고 보면 된다.
    # train_input_enc, train_target_dec_length, train_target_dec 
    # 3개를 각각 한문장으로 나눈다.
    dataset = tf.data.Dataset.from_tensor_slices((train_input_enc, train_target_dec_length, train_target_dec))
    # 전체 데이터를 썩는다.
    dataset = dataset.shuffle(buffer_size=len(train_input_enc))
    # 배치 인자 값이 없다면  에러를 발생 시킨다.
    assert batch_size is not None, "train batchSize must not be None"
    # from_tensor_slices를 통해 나눈것을 
    # 배치크기 만큼 묶어 준다.
    dataset = dataset.batch(batch_size)
    # 데이터 각 요소에 대해서 train_rearrange 함수를 
    # 통해서 요소를 변환하여 맵으로 구성한다.
    dataset = dataset.map(train_rearrange)
    # repeat()함수에 원하는 에포크 수를 넣을수 있으면 
    # 아무 인자도 없다면 무한으로 이터레이터 된다.
    dataset = dataset.repeat()
    # make_one_shot_iterator를 통해 이터레이터를 
    # 만들어 준다.
    iterator = dataset.make_one_shot_iterator()
    # 이터레이터를 통해 다음 항목의 텐서 
    # 개체를 넘겨준다.
    return iterator.get_next()


In [14]:
# 평가에 들어가 배치 데이터를 만드는 함수이다.
def eval_input_fn(eval_input_enc, eval_target_dec, batch_size):
    # Dataset을 생성하는 부분으로써 from_tensor_slices부분은 
    # 각각 한 문장으로 자른다고 보면 된다.
    # eval_input_enc, eval_target_dec, batch_size 
    # 3개를 각각 한문장으로 나눈다.
    dataset = tf.data.Dataset.from_tensor_slices((eval_input_enc, eval_target_dec))
    # 전체 데이터를 섞는다.
    dataset = dataset.shuffle(buffer_size=len(eval_input_enc))
    # 배치 인자 값이 없다면  에러를 발생 시킨다.
    assert batch_size is not None, "eval batchSize must not be None"
    # from_tensor_slices를 통해 나눈것을 
    # 배치크기 만큼 묶어 준다.
    dataset = dataset.batch(batch_size)
    # 데이터 각 요소에 대해서 rearrange 함수를 
    # 통해서 요소를 변환하여 맵으로 구성한다.
    dataset = dataset.map(rearrange)
    # repeat()함수에 원하는 에포크 수를 넣을수 있으면 
    # 아무 인자도 없다면 무한으로 이터레이터 된다.
    # 평가이므로 1회만 동작 시킨다.
    dataset = dataset.repeat(1)
    # make_one_shot_iterator를 통해 
    # 이터레이터를 만들어 준다.
    iterator = dataset.make_one_shot_iterator()
    # 이터레이터를 통해 다음 항목의 
    # 텐서 개체를 넘겨준다.
    return iterator.get_next()

In [15]:
def data_tokenizer(data):
    # 토크나이징 해서 담을 배열 생성
    words = []
    for sentence in data:
        # FILTERS = "([~.,!?\"':;)(])"
        # 위 필터와 같은 값들을 정규화 표현식을 
        # 통해서 모두 "" 으로 변환 해주는 부분이다.
        sentence = re.sub(CHANGE_FILTER, "", sentence)
        for word in sentence.split():
            words.append(word)
    # 토그나이징과 정규표현식을 통해 만들어진 
    # 값들을 넘겨 준다.
    return [word for word in words if word]

In [16]:
def load_vocabulary():
    # 사전을 담을 배열 준비한다.
    vocabulary_list = []
    # 사전을 구성한 후 파일로 저장 진행한다. 
    # 그 파일의 존재 유무를 확인한다.
    if (not (os.path.exists(DEFINES.vocabulary_path))):
        # 이미 생성된 사전 파일이 존재하지 않으므로 
        # 데이터를 가지고 만들어야 한다.
        # 그래서 데이터가 존재 하면 사전을 만들기 위해서 
        # 데이터 파일의 존재 유무를 확인한다.
        if (os.path.exists(DEFINES.data_path)):
            # 데이터가 존재하니 판단스를 통해서 
            # 데이터를 불러오자
            data_df = pd.read_csv(DEFINES.data_path, encoding='utf-8')
            # 판다스의 데이터 프레임을 통해서 
            # 질문과 답에 대한 열을 가져 온다.
            question, answer = list(data_df['Q']), list(data_df['A'])
            if DEFINES.tokenize_as_morph:  # 형태소에 따른 토크나이져 처리
                question = prepro_like_morphlized(question)
                answer = prepro_like_morphlized(answer)
            data = []
            # 질문과 답변을 extend을 
            # 통해서 구조가 없는 배열로 만든다.
            data.extend(question)
            data.extend(answer)
            # 토큰나이져 처리 하는 부분이다.
            words = data_tokenizer(data)
            # 공통적인 단어에 대해서는 모두 
            # 필요 없으므로 한개로 만들어 주기 위해서
            # set해주고 이것들을 리스트로 만들어 준다.
            words = list(set(words))
            # 데이터 없는 내용중에 MARKER를 사전에 
            # 추가 하기 위해서 아래와 같이 처리 한다.
            # 아래는 MARKER 값이며 리스트의 첫번째 부터 
            # 순서대로 넣기 위해서 인덱스 0에 추가한다.
            # PAD = "<PADDING>"
            # STD = "<START>"
            # END = "<END>"
            # UNK = "<UNKNOWN>"     
            words[:0] = MARKER
        # 사전을 리스트로 만들었으니 이 내용을 
        # 사전 파일을 만들어 넣는다.
        with open(DEFINES.vocabulary_path, 'w', encoding='utf-8') as vocabulary_file:
            for word in words:
                vocabulary_file.write(word + '\n')

    # 사전 파일이 존재하면 여기에서 
    # 그 파일을 불러서 배열에 넣어 준다.
    with open(DEFINES.vocabulary_path, 'r', encoding='utf-8') as vocabulary_file:
        for line in vocabulary_file:
            vocabulary_list.append(line.strip())

    # 배열에 내용을 키와 값이 있는 
    # 딕셔너리 구조로 만든다.
    char2idx, idx2char = make_vocabulary(vocabulary_list)
    # 두가지 형태의 키와 값이 있는 형태를 리턴한다. 
    # (예) 단어: 인덱스 , 인덱스: 단어)
    return char2idx, idx2char, len(char2idx)

In [17]:
def make_vocabulary(vocabulary_list):
    # 리스트를 키가 단어이고 값이 인덱스인 
    # 딕셔너리를 만든다.
    char2idx = {char: idx for idx, char in enumerate(vocabulary_list)}
    # 리스트를 키가 인덱스이고 값이 단어인 
    # 딕셔너리를 만든다.
    idx2char = {idx: char for idx, char in enumerate(vocabulary_list)}
    # 두개의 딕셔너리를 넘겨 준다.
    return char2idx, idx2char

In [18]:
def main_data(self):
    char2idx, idx2char, vocabulary_length = load_vocabulary()

# model.py

In [36]:
# -*- coding: utf-8 -*-
#import tensorflow as tf
import sys
import numpy as np
#from configs import DEFINES

In [37]:
def make_lstm_cell(mode, hiddenSize, index):
    cell = tf.nn.rnn_cell.BasicLSTMCell(hiddenSize, name="lstm" + str(index), state_is_tuple=False)
    if mode == tf.estimator.ModeKeys.TRAIN:
        cell = tf.nn.rnn_cell.DropoutWrapper(cell, state_keep_prob=DEFINES.dropout_width) #tf.contrib.rnn.DropoutWrapper()
    return cell

In [38]:
# 에스티메이터 모델 부분이다.
# freatures : tf.data.Dataset.map을 통해서 만들어진 
# features = {"input": input, "length": length}
# labels : tf.data.Dataset.map을 통해서 만들어진 target
# mode는 에스티메이터 함수를 호출하면 에스티메이터 
# 프레임워크 모드의 값이 해당 부분이다.
# params : 에스티메이터를 구성할때 params 값들이다. 
# (params={ # 모델 쪽으로 파라메터 전달한다.)
def Model(features, labels, mode, params):
    TRAIN = mode == tf.estimator.ModeKeys.TRAIN
    EVAL = mode == tf.estimator.ModeKeys.EVAL
    PREDICT = mode == tf.estimator.ModeKeys.PREDICT

    # 미리 정의된  임베딩 사용 유무를 확인 한다.
    # 값이 True이면 임베딩을 해서 학습하고 False이면 
    # onehotencoding 처리 한다.
    if params['embedding'] == True:
        # 가중치 행렬에 대한 초기화 함수이다.
        # xavier (Xavier Glorot와 Yoshua Bengio (2010)
        # URL : http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf
        initializer = tf.keras.initializers.glorot_uniform() #tf.contrib.layers.xavier_initializer()
        # 인코딩 변수를 선언하고 값을 설정한다.
        embedding_encoder = tf.get_variable(name="embedding_encoder",  # 이름
                                           shape=[params['vocabulary_length'], params['embedding_size']],  # 모양
                                           dtype=tf.float32,  # 타입
                                           initializer=initializer,  # 초기화 값
                                           trainable=True)  # 학습 유무
    else:
        # tf.eye를 통해서 사전의 크기 만큼의 단위행렬 
        # 구조를 만든다.
        embedding_encoder = tf.eye(num_rows=params['vocabulary_length'], dtype=tf.float32)
        # 인코딩 변수를 선언하고 값을 설정한다.
        embedding_encoder = tf.get_variable(name="embedding_encoder",  # 이름
                                           initializer=embedding_encoder,  # 초기화 값
                                           trainable=False)  # 학습 유무

    # embedding_lookup을 통해서 features['input']의 인덱스를
    # 위에서 만든 embedding_encoder의 인덱스의 값으로 변경하여 
    # 임베딩된 디코딩 배치를 만든다.
    embedding_encoder_batch = tf.nn.embedding_lookup(params=embedding_encoder, ids=features['input'])

    # 미리 정의된  임베딩 사용 유무를 확인 한다.
    # 값이 True이면 임베딩을 해서 학습하고 False이면 
    # onehotencoding 처리 한다.
    if params['embedding'] == True:
        # 가중치 행렬에 대한 초기화 함수이다.
        # xavier (Xavier Glorot와 Yoshua Bengio (2010)
        # URL : http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf
        initializer = tf.keras.initializers.glorot_uniform() #tf.contrib.layers.xavier_initializer()
        # 디코딩 변수를 선언하고 값을 설정한다.
        embedding_decoder = tf.get_variable(name="embedding_decoder",  # 이름
                                           shape=[params['vocabulary_length'], params['embedding_size']],  # 모양
                                           dtype=tf.float32,  # 타입
                                           initializer=initializer,  # 초기화 값
                                           trainable=True)  # 학습 유무
    else:
        # tf.eye를 통해서 사전의 크기 만큼의 단위행렬 
        # 구조를 만든다.
        embedding_decoder = tf.eye(num_rows=params['vocabulary_length'], dtype=tf.float32)
        # 인코딩 변수를 선언하고 값을 설정한다.
        embedding_decoder = tf.get_variable(name='embedding_decoder',  # 이름
                                           initializer=embedding_decoder,  # 초기화 값
                                           trainable=False)  # 학습 유무

    # 변수 재사용을 위해서 reuse=.AUTO_REUSE를 사용하며 범위를
    # 정해주고 사용하기 위해 scope설정을 한다.
    # make_lstm_cell이 "cell"반복적으로 호출 되면서 재사용된다.
    with tf.variable_scope('encoder_scope', reuse=tf.AUTO_REUSE):
        # 값이 True이면 멀티레이어로 모델을 구성하고 False이면 
        # 단일레이어로 모델을 구성 한다.
        if params['multilayer'] == True:
            # layerSize 만큼  LSTMCell을  encoder_cell_list에 담는다.
            encoder_cell_list = [make_lstm_cell(mode, params['hidden_size'], i) for i in range(params['layer_size'])]
            # MUltiLayer RNN CEll에 encoder_cell_list를 넣어 멀티 레이어를 만든다.
            rnn_cell = tf.nn.rnn_cell.MultiRNNCell(encoder_cell_list, state_is_tuple=False) #tf.contrib.rnn.MultiRNNCell
        else: 
            # 단층 LSTMLCell을 만든다.
            rnn_cell = make_lstm_cell(mode, params['hidden_size'], "")
        # rnn_cell에 의해 지정된 반복적인 신경망을 만든다.
        # encoder_outputs(RNN 출력 Tensor)[batch_size, 
        # max_time, cell.output_size]
        # encoder_states 최종 상태  [batch_size, cell.state_size]
        encoder_outputs, encoder_states = tf.nn.dynamic_rnn(cell=rnn_cell,  # RNN 셀
                                                              inputs=embedding_encoder_batch,  # 입력 값
                                                              dtype=tf.float32)  # 타입
        # 변수 재사용을 위해서 reuse=.AUTO_REUSE를 사용하며 범위를 정해주고
        # 사용하기 위해 scope설정을 한다.
        # make_lstm_cell이 "cell"반복적으로 호출 되면서 재사용된다.
    with tf.variable_scope('decoder_scope', reuse=tf.AUTO_REUSE):
        # 값이 True이면 멀티레이어로 모델을 구성하고 False이면 단일레이어로
        # 모델을 구성 한다.
        if params['multilayer'] == True:
            # layer_size 만큼  LSTMCell을  decoder_cell_list에 담는다.
            decoder_cell_list = [make_lstm_cell(mode, params['hidden_size'], i) for i in range(params['layer_size'])]
            # MUltiLayer RNN CEll에 decoder_cell_list를 넣어 멀티 레이어를 만든다.
            rnn_cell = tf.nn.rnn_cell.MultiRNNCell(decoder_cell_list, state_is_tuple=False) #tf.contrib.rnn.MultiRNNCell
        else:
            # 단층 LSTMLCell을 만든다.
            rnn_cell = make_lstm_cell(mode, params['hidden_size'], "")

        decoder_state = encoder_states
        # 매 타임 스텝에 나오는 아웃풋을 저장하는 리스트 두개를 만든다. 
        # 하나는 토큰 인덱스는 predict_tokens 저장
        # 다른 하나는 temp_logits에 logits 저장한다.
        predict_tokens = list()
        temp_logits = list()

        # 평가인 경우에는 teacher forcing이 되지 않도록 해야한다.
        # 따라서 학습이 아닌경우에 is_train을 False로 하여 teacher forcing이 되지 않도록 한다.
        output_token = tf.ones(shape=(tf.shape(encoder_outputs)[0],), dtype=tf.int32) * 1
        # 전체 문장 길이 만큼 타임 스텝을 돌도록 한다.
        for i in range(DEFINES.max_sequence_length):
            # 두 번쨰 스텝 이후에는 teacher forcing을 적용하는지 확률에 따라 결정하도록 한다.
            # teacher forcing rate은 teacher forcing을 어느정도 줄 것인지를 조절한다.
            if TRAIN:
                if i > 0:
                    # tf.cond를 통해 rnn에 입력할 입력 임베딩 벡터를 결정한다 여기서 true인 경우엔 입력된 output값 아닌경우에는 이전 스텝에
                    # 나온 output을 사용한다.
                    input_token_emb = tf.cond(
                        tf.logical_and( # 논리 and 연산자
                            True,
                            tf.random_uniform(shape=(), maxval=1) <= params['teacher_forcing_rate'] # 률에 따른 labels값 지원 유무
                        ),
                        lambda: tf.nn.embedding_lookup(embedding_decoder, labels[:, i-1]),  # labels 정답을 넣어주고 있다.
                        lambda: tf.nn.embedding_lookup(embedding_decoder, output_token) # 모델이 정답이라고 생각 하는 값
                    )
                else:
                    input_token_emb = tf.nn.embedding_lookup(embedding_decoder, output_token) # 모델이 정답이라고 생각 하는 값
            else: # 평가 및 예측은 여기를 진행해야 한다. 
                input_token_emb = tf.nn.embedding_lookup(embedding_decoder, output_token)

            # 어텐션 적용 부분
            if params['attention'] == True:
                W1 = tf.keras.layers.Dense(params['hidden_size'])
                W2 = tf.keras.layers.Dense(params['hidden_size'])
                V = tf.keras.layers.Dense(1)
                # (?, 256) -> (?, 128)
                hidden_with_time_axis = W2(decoder_state)
                # (?, 128) -> (?, 1, 128)
                hidden_with_time_axis = tf.expand_dims(hidden_with_time_axis, axis=1)
                # (?, 1, 128) -> (?, 25, 128)
                hidden_with_time_axis = tf.manip.tile(hidden_with_time_axis, [1, DEFINES.max_sequence_length, 1])
                # (?, 25, 1)
                score = V(tf.nn.tanh(W1(encoder_outputs) + hidden_with_time_axis))
                # score = V(tf.nn.tanh(W1(encoderOutputs) + tf.manip.tile(tf.expand_dims(W2(decoder_state), axis=1), [1, DEFINES.maxSequenceLength, 1])))
                # (?, 25, 1)
                attention_weights = tf.nn.softmax(score, axis=-1)
                # (?, 25, 128)
                context_vector = attention_weights * encoder_outputs
                # (?, 25, 128) -> (?, 128)
                context_vector = tf.reduce_sum(context_vector, axis=1)
                # (?, 256)
                input_token_emb = tf.concat([context_vector, input_token_emb], axis=-1)

            # RNNCell을 호출하여 RNN 스텝 연산을 진행하도록 한다.
            input_token_emb = tf.keras.layers.Dropout(0.5)(input_token_emb)
            decoder_outputs, decoder_state = rnn_cell(input_token_emb, decoder_state)
            decoder_outputs = tf.keras.layers.Dropout(0.5)(decoder_outputs)
            # feedforward를 거쳐 output에 대한 logit값을 구한다.
            output_logits = tf.layers.dense(decoder_outputs, params['vocabulary_length'], activation=None)

            # softmax를 통해 단어에 대한 예측 probability를 구한다.
            output_probs = tf.nn.softmax(output_logits)
            output_token = tf.argmax(output_probs, axis=-1)

            # 한 스텝에 나온 토큰과 logit 결과를 저장해둔다.
            predict_tokens.append(output_token)
            temp_logits.append(output_logits)

        # 저장했던 토큰과 logit 리스트를 stack을 통해 메트릭스로 만들어 준다.
        # 만들게 뙤면 차원이 [시퀀스 X 배치 X 단어 feature 수] 이렇게 되는데
        # 이를 transpose하여 [배치 X 시퀀스 X 단어 feature 수] 로 맞춰준다.
        predict = tf.transpose(tf.stack(predict_tokens, axis=0), [1, 0])
        logits = tf.transpose(tf.stack(temp_logits, axis=0), [1, 0, 2])

        print(predict.shape)
        print(logits.shape)

    if PREDICT:
        if params['serving'] == True:
            export_outputs = {
                'indexs': tf.estimator.export.PredictOutput(predict) # 서빙 결과값을 준다.
            }

        predictions = {  # 예측 값들이 여기에 딕셔너리 형태로 담긴다.
            'indexs': predict,  # 시퀀스 마다 예측한 값
            'logits': logits,  # 마지막 결과 값
        }
        # 에스티메이터에서 리턴하는 값은 tf.estimator.EstimatorSpec 
        # 객체를 리턴 한다.
        # mode : 에스티메이터가 수행하는 mode (tf.estimator.ModeKeys.PREDICT)
        # predictions : 예측 값
        if params['serving'] == True:
            return tf.estimator.EstimatorSpec(mode, predictions=predictions, export_outputs=export_outputs)

        return tf.estimator.EstimatorSpec(mode, predictions=predictions)

    # 마지막 결과 값과 정답 값을 비교하는 
    # tf.nn.sparse_softmax_cross_entropy_with_logits(로스함수)를 
    # 통과 시켜 틀린 만큼의
    # 에러 값을 가져 오고 이것들은 차원 축소를 통해 단일 텐서 값을 반환 한다.
    # pad의 loss값을 무력화 시킨다. pad가 아닌값은 1 pad인 값은 0을 주어 동작
    # 하도록 한다.
    # 정답 차원 변경을 한다. [배치 * max_sequence_length * vocabulary_length]  
    # logits과 같은 차원을 만들기 위함이다.
    labels_ = tf.one_hot(labels, params['vocabulary_length'])
    
    if TRAIN and params['loss_mask'] == True:
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=labels_))
        masks = features['length']

        loss = loss * tf.cast(masks, tf.float32)
        loss = tf.reduce_mean(loss)
    else:
       loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=labels_))
    # 라벨과 결과가 일치하는지 빈도 계산을 통해 
    # 정확도를 측정하는 방법이다.
    accuracy = tf.metrics.accuracy(labels=labels, predictions=predict, name='accOp')

    # 정확도를 전체값으로 나눈 값이다.
    metrics = {'accuracy': accuracy}
    tf.summary.scalar('accuracy', accuracy[1])

    # 평가 mode 확인 부분이며 평가는 여기 까지 
    # 수행하고 리턴한다.
    if EVAL:
        # 에스티메이터에서 리턴하는 값은 
        # tf.estimator.EstimatorSpec 객체를 리턴 한다.
        # mode : 에스티메이터가 수행하는 mode (tf.estimator.ModeKeys.EVAL)
        # loss : 에러 값
        # eval_metric_ops : 정확도 값
        return tf.estimator.EstimatorSpec(mode, loss=loss, eval_metric_ops=metrics)

    # 파이썬 assert구문으로 거짓일 경우 프로그램이 종료 된다.
    # 수행 mode(tf.estimator.ModeKeys.TRAIN)가 
    # 아닌 경우는 여기 까지 오면 안되도록 방어적 코드를 넣은것이다.
    assert TRAIN

    # 아담 옵티마이저를 사용한다.
    optimizer = tf.train.AdamOptimizer(learning_rate=DEFINES.learning_rate)
    # 에러값을 옵티마이저를 사용해서 최소화 시킨다.
    train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())
    # 에스티메이터에서 리턴하는 값은 tf.estimator.EstimatorSpec 객체를 리턴 한다.
    # mode : 에스티메이터가 수행하는 mode (tf.estimator.ModeKeys.EVAL)
    # loss : 에러 값
    # train_op : 그라디언트 반환
    return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)

# main.py

In [39]:
#import tensorflow as tf
#import model as ml
#import data
import numpy as np
import os
import sys
#from configs import DEFINES

In [40]:
# Serving 기능을 위하여 serving 함수를 구성한다.
def serving_input_receiver_fn():
    receiver_tensor = {
        'input': tf.placeholder(dtype=tf.int32, shape=[None, DEFINES.max_sequence_length]),
        'output': tf.placeholder(dtype=tf.int32, shape=[None, DEFINES.max_sequence_length])    
    }
    features = {
        key: tensor for key, tensor in receiver_tensor.items()
    }
    return tf.estimator.export.ServingInputReceiver(features, receiver_tensor)

In [41]:
DATA_OUT_PATH = './data_out/'
data_out_path = os.path.join(os.getcwd(), DATA_OUT_PATH)
os.makedirs(data_out_path, exist_ok=True)

In [42]:
# 데이터를 통한 사전 구성 한다.
char2idx, idx2char, vocabulary_length = load_vocabulary() #data.load_vocabulary()

In [43]:
char2idx

{'<PADDING>': 0,
 '<START>': 1,
 '<END>': 2,
 '<UNKNOWN>': 3,
 '힘들다고': 4,
 '조금만': 5,
 '아니어도': 6,
 '싫을': 7,
 '정리했길': 8,
 '중요': 9,
 '보장해요': 10,
 '있자고': 11,
 '전문가를': 12,
 '싫대': 13,
 '우울하고': 14,
 '열흘짼데': 15,
 '우산빌려달라고': 16,
 '될런지': 17,
 '넓어': 18,
 '차단시': 19,
 '두면': 20,
 '걸렸어': 21,
 '평범하면서': 22,
 '안정적이고': 23,
 '내려놓을': 24,
 '수고했어요': 25,
 '나쁜데': 26,
 '가수의': 27,
 '봤다고': 28,
 '따듯해지니까': 29,
 '연애했더니': 30,
 '힘들거예요': 31,
 '삼재인가봐요': 32,
 '성장하게하죠': 33,
 '지켜달라고': 34,
 '키워': 35,
 '학과': 36,
 '사실이라면': 37,
 '1달': 38,
 '안할생각입니다': 39,
 '커지거나': 40,
 '친절하게': 41,
 '세달': 42,
 '선물이': 43,
 '옥상에': 44,
 '라는말에': 45,
 '팔': 46,
 '들려주세요': 47,
 '고집부렸을까': 48,
 '잘해주는데': 49,
 '표현한다는게': 50,
 '생일전에': 51,
 '겠다': 52,
 '체질이': 53,
 '아물런지': 54,
 '본다면': 55,
 '극복할수있을까': 56,
 '부여하는게': 57,
 '보셨나봐요': 58,
 '쏟아붓고': 59,
 '서류에서': 60,
 '모르겠네요': 61,
 '많터냐': 62,
 '깍고': 63,
 '한통': 64,
 '위층': 65,
 '제자리여도': 66,
 '사적': 67,
 '이런사진을': 68,
 '넣어두세요': 69,
 '공부해서': 70,
 '자취': 71,
 '지르는': 72,
 '옷보러가자고': 73,
 '느끼나봐요': 74,
 '1년됐어': 75,
 '다가왔어': 76,
 '졸업해

In [44]:
# 훈련 데이터와 테스트 데이터를 가져온다.
train_input, train_label, eval_input, eval_label = load_data() #data.load_data()

In [45]:
train_label

['여기까지 인연이었나봅니다.',
 '잘했어요.',
 '싫은 게 통해야 편하죠.',
 '사귀기 전에 마음을 확인하는 단계라서 그렇지 않을까요?',
 '궁금할 수 있어요.',
 '좋은 분이시군요',
 '맛나게 끓여드세요.',
 '주변 사람들에게 부탁해보세요.',
 '엄마께 프라이버시를 지켜달라고 말해보세요.',
 '가족들과 상의해보세요.',
 '조금만 더 버텨보세요.',
 '서운한 마음을 충분히 전하는게 좋겠어요.',
 '누구든 힘들어할 거예요.',
 '서로를 아는 좋은 기회가 되겠네요.',
 '썸 타다가 발전되지 못한 관계죠.',
 '학원을 다니거나 연습하면 잘할 수 있을 거예요.',
 '사람 마을은 알기 힘들어요.',
 '그게 인생이죠.',
 '그 과정을 통해 성장할 수 있을 거예요.',
 '눈을 감고 명상을 해보세요.',
 '부럽네요.',
 '이별하면 연락 안 하는게 맞아요.',
 'CC는 아무래도 제약이 있을 거예요.',
 '남과 비교하지 마세요.',
 '참 행복한가봐요.',
 '헤어짐에도 처방약이 있으면 좋겠어요.',
 '안녕.',
 '신고하고 차단해보세요.',
 '철은 죽을 때 들어도 돼요.',
 '어쨌든 일이니까요.',
 '마음이 조급해지면 하지 않을 실수를 하게 돼요.',
 '잘 알아보고 사세요.',
 '좋은 이별은 없어요.',
 '기억에 취해서 그럴 거예요.',
 '나들이 가보세요.',
 '눈에서 꿀이 떨어지네요.',
 '거울을 보면서 얼굴 망가뜨리기 놀이 해 보신 적 있으세요? 의외로 재미 있어요.',
 '좋았겠어요.',
 '맛나게 드세요.',
 '주말 저녁에 약속을 잡아보세요.',
 '좋지 않을거라 생각해요.',
 '많이 지쳤나봐요.',
 '자신만 생각하는 사람은 멀리하세요.',
 '그럼 새로운 인연을 기다려보세요.',
 '많이 좋아했던 사람인가봐요.',
 '생각이 너무 많아서 그런 것 같습니다.',
 '이젠 잊어버리세요. 미련은 독이 됩니다.',
 '맘처럼 할 수 없는 부분 같아요.',
 '확인해달라고 해보세요.',
 '저는 위로봇입니다.'

In [46]:
# 훈련셋 인코딩 만드는 부분이다.
train_input_enc, train_input_enc_length = enc_processing(train_input, char2idx) #data.enc_processing()

In [47]:
# 훈련셋 디코딩 출력 부분 만드는 부분이다.
train_target_dec, train_target_dec_length = dec_target_processing(train_label, char2idx) #data.dec_target_processing()

In [48]:
# 평가셋 인코딩 만드는 부분이다.
eval_input_enc, eval_input_enc_length = enc_processing(eval_input,char2idx) #data.enc_processing()

In [49]:
# 평가셋 디코딩 출력 부분 만드는 부분이다.
eval_target_dec, _ = dec_target_processing(eval_label, char2idx) #data.dec_target_processing

In [50]:
# 현재 경로'./'에 현재 경로 하부에 
# 체크 포인트를 저장한 디렉토리를 설정한다.
check_point_path = os.path.join(os.getcwd(), DEFINES.check_point_path)
save_model_path = os.path.join(os.getcwd(), DEFINES.save_model_path)
# 디렉토리를 만드는 함수이며 두번째 인자 exist_ok가 
# True이면 디렉토리가 이미 존재해도 OSError가 
# 발생하지 않는다.
# exist_ok가 False이면 이미 존재하면 
# OSError가 발생한다.
os.makedirs(check_point_path, exist_ok=True)
os.makedirs(save_model_path, exist_ok=True)

In [51]:
# 에스티메이터 구성한다.
classifier = tf.estimator.Estimator(
    model_fn=Model,  # 모델 등록한다. #ml.Model,
    model_dir=DEFINES.check_point_path,  # 체크포인트 위치 등록한다.
    params={  # 모델 쪽으로 파라메터 전달한다.
        'hidden_size': DEFINES.hidden_size,  # 가중치 크기 설정한다.
        'layer_size': DEFINES.layer_size,  # 멀티 레이어 층 개수를 설정한다.
        'learning_rate': DEFINES.learning_rate,  # 학습율 설정한다.
        'teacher_forcing_rate': DEFINES.teacher_forcing_rate, # 학습시 디코더 인풋 정답 지원율 설정
        'vocabulary_length': vocabulary_length,  # 딕셔너리 크기를 설정한다.
        'embedding_size': DEFINES.embedding_size,  # 임베딩 크기를 설정한다.
        'embedding': DEFINES.embedding,  # 임베딩 사용 유무를 설정한다.
        'multilayer': DEFINES.multilayer,  # 멀티 레이어 사용 유무를 설정한다.
        'attention': DEFINES.attention, #  어텐션 지원 유무를 설정한다.
        'teacher_forcing': DEFINES.teacher_forcing, # 학습시 디코더 인풋 정답 지원 유무 설정한다.
        'loss_mask': DEFINES.loss_mask, # PAD에 대한 마스크를 통한 loss를 제한 한다.
        'serving': DEFINES.serving # 모델 저장 및 serving 유무를 설정한다.
    })

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './data_out/check_point', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1a3a8fd090>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [52]:
# 학습 실행
# data.train_input_fn()
classifier.train(input_fn=lambda: train_input_fn(
    train_input_enc, train_target_dec_length, train_target_dec, DEFINES.batch_size), steps=DEFINES.train_steps)

INFO:tensorflow:Calling model_fn.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.
(?, 25)
(?, 25, 20705)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done 

KeyboardInterrupt: 

In [ ]:
# 서빙 기능 유무에 따라 모델을 Save 한다.
if DEFINES.serving == True:
    save_model_path = classifier.export_savedmodel(
        export_dir_base=DEFINES.save_model_path,
        serving_input_receiver_fn=serving_input_receiver_fn)

In [ ]:
# 평가 실행
# data.eval_input_fn()
eval_result = classifier.evaluate(input_fn=lambda: eval_input_fn(
    eval_input_enc,eval_target_dec, DEFINES.batch_size))
print('\nEVAL set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

# predict.py

In [ ]:
"""
#import tensorflow as tf
#import data
import sys
#import model as ml
#from configs import DEFINES
	
if __name__ == '__main__':
    tf.logging.set_verbosity(tf.logging.INFO)
    arg_length = len(sys.argv)
    
    if(arg_length < 2):
        raise Exception("Don't call us. We'll call you")
  
    
    # 데이터를 통한 사전 구성 한다.
    char2idx,  idx2char, vocabulary_length = data.load_vocabulary()

    # 테스트용 데이터 만드는 부분이다.
    # 인코딩 부분 만든다.
    input = ""
    for i in sys.argv[1:]:
        input += i 
        input += " "
        
    print(input)
    predic_input_enc, predic_input_enc_length = data.enc_processing([input], char2idx)
    # 학습 과정이 아니므로 디코딩 입력은 
    # 존재하지 않는다.(구조를 맞추기 위해 넣는다.)
    # 학습 과정이 아니므로 디코딩 출력 부분도 
    # 존재하지 않는다.(구조를 맞추기 위해 넣는다.)
    predic_target_dec, _ = data.dec_target_processing([""], char2idx)

    if DEFINES.serving == True:
        # 모델이 저장된 위치를 넣어 준다.  export_dir
        predictor_fn = tf.contrib.predictor.from_saved_model(
            export_dir="/home/evo_mind/DeepLearning/NLP/Work/ChatBot2_Final/data_out/model/1541575161"
        )
    else:
        # 에스티메이터 구성한다.
        classifier = tf.estimator.Estimator(
                model_fn=ml.Model, # 모델 등록한다.
                model_dir=DEFINES.check_point_path, # 체크포인트 위치 등록한다.
                params={ # 모델 쪽으로 파라메터 전달한다.
                    'hidden_size': DEFINES.hidden_size,  # 가중치 크기 설정한다.
                    'layer_size': DEFINES.layer_size,  # 멀티 레이어 층 개수를 설정한다.
                    'learning_rate': DEFINES.learning_rate,  # 학습율 설정한다.
                    'teacher_forcing_rate': DEFINES.teacher_forcing_rate, # 학습시 디코더 인풋 정답 지원율 설정
                    'vocabulary_length': vocabulary_length,  # 딕셔너리 크기를 설정한다.
                    'embedding_size': DEFINES.embedding_size,  # 임베딩 크기를 설정한다.
                    'embedding': DEFINES.embedding,  # 임베딩 사용 유무를 설정한다.
                    'multilayer': DEFINES.multilayer,  # 멀티 레이어 사용 유무를 설정한다.
                    'attention': DEFINES.attention, #  어텐션 지원 유무를 설정한다.
                    'teacher_forcing': DEFINES.teacher_forcing, # 학습시 디코더 인풋 정답 지원 유무 설정한다.
                    'loss_mask': DEFINES.loss_mask, # PAD에 대한 마스크를 통한 loss를 제한 한다.
                    'serving': DEFINES.serving # 모델 저장 및 serving 유무를 설정한다.
                })

    if DEFINES.serving == True:
        predictions = predictor_fn({'input':predic_input_enc, 'output':predic_target_dec})
        data.pred2string(predictions, idx2char)
    else:
        # 예측을 하는 부분이다.
        predictions = classifier.predict(
            input_fn=lambda:data.eval_input_fn(predic_input_enc, predic_target_dec, DEFINES.batch_size))
        # 예측한 값을 인지 할 수 있도록 
        # 텍스트로 변경하는 부분이다.
        data.pred2string(predictions, idx2char)
"""